## API Test Notebook

In [1]:
import requests
import json
import pandas as pd

# Base URL for the Flask API
BASE_URL = "http://127.0.0.1:5000"

print("🚀 Starting API Tests...")
print(f"Base URL: {BASE_URL}")

🚀 Starting API Tests...
Base URL: http://127.0.0.1:5000


In [2]:
# Test health endpoint
try:
    response = requests.get(f"{BASE_URL}/health")
    print(f"✅ Health Check: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"❌ Health Check Failed: {e}")

✅ Health Check: 200
Response: {'model_loaded': True, 'status': 'healthy', 'timestamp': 'BlackScholesCalculator available'}


In [7]:
# Test sample endpoint
try:
    response = requests.get(f"{BASE_URL}/sample")
    print(f"✅ Sample: {response.status_code}")
    data = response.json()
    print(f"Sample prediction: ${data['prediction']:.4f}")
    print(f"Input: {data['input_data']}")
except Exception as e:
    print(f"❌ Sample Failed: {e}")

✅ Sample: 200
Sample prediction: $-27.9201
Input: {'contract_type': 'call', 'implied_volatility': 0.25, 'strike': 455.0, 'time_to_expiry': 0.0833, 'underlying_price': 450.0}


In [8]:
# Test custom prediction
test_option = {
    "strike": 150.0,
    "implied_volatility": 0.25,
    "open_interest": 1000,
    "volume": 50,
    "underlying_price": 155.0,
    "risk_free_rate": 0.05,
    "time_to_expiry": 0.25,
    "contract_type": "call"
}

try:
    response = requests.post(f"{BASE_URL}/predict", json=test_option)
    print(f"✅ Custom Prediction: {response.status_code}")
    result = response.json()
    print(f"Predicted Error: ${result['prediction']:.4f}")
    print(f"Black-Scholes Price: ${result['black_scholes_price']:.2f}")
except Exception as e:
    print(f"❌ Custom Prediction Failed: {e}")

✅ Custom Prediction: 200
Predicted Error: $-27.7809
Black-Scholes Price: $11.47


In [9]:
# Test multiple strike prices
strikes = [140, 150, 160, 170]
results = []

for strike in strikes:
    test_data = test_option.copy()
    test_data['strike'] = strike
    
    try:
        response = requests.post(f"{BASE_URL}/predict", json=test_data)
        if response.status_code == 200:
            result = response.json()
            results.append({
                'strike': strike,
                'prediction': result['prediction'],
                'bs_price': result['black_scholes_price']
            })
    except:
        continue

# Show results
df = pd.DataFrame(results)
print("📊 Strike Price Analysis:")
print(df)

📊 Strike Price Analysis:
   strike  prediction   bs_price
0     140  -27.780927  18.527195
1     150  -27.780854  11.474817
2     160  -27.678726   6.385485
3     170  -27.678726   3.187234


In [10]:
# Test invalid input
invalid_data = {"strike": "invalid", "volume": -10}

try:
    response = requests.post(f"{BASE_URL}/predict", json=invalid_data)
    print(f"Error Test Status: {response.status_code}")
    print(f"Error Response: {response.json()}")
except Exception as e:
    print(f"Error handling test: {e}")

Error Test Status: 400
Error Response: {'error': 'Missing required field: underlying_price'}
